In [1]:
# matplotlib에서 한글 폰트를 설정하는 방법
import matplotlib.pyplot as plt
plt.rc("font", family="malgun gothic")

# 음수 기호 출력 방법
import matplotlib
matplotlib.rcParams["axes.unicode_minus"]=False

# 그래프를 선명하게 출력하는 방법
%config InlineBackend.figure_format = "retina"

import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Q1) 데이터 로드 

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd

# step 1. 데이터 준비
housing = fetch_california_housing()

X=pd.DataFrame(housing.data, columns = housing.feature_names) # 설명변수  
y=pd.DataFrame(housing.target, columns=['Price'])  #목적변수 (정답 레이블)

In [3]:
#2.  훈련셋트랑 테스로 세트로 분할  : 데이터셋 20%를 테스트데이터로 분할
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size =0.2 , random_state =0)
X_train.shape, X_test.shape

((16512, 8), (4128, 8))

In [4]:
#3.설명변수와, 목적변수를 pandas 데이터 프레임에 저장 하자.  
train_set = pd.concat([X_train,y_train], axis=1)
test_set = pd.concat([X_test, y_test], axis=1)

In [5]:
#4.train_set[학습용] , test_set[테스트 데이터 셋]  의 각 통계 정보를 확인  
train_set.describe()
#test_set.describe()
#[생각] 테스트 셋 데이터는 알려지지 않은 데이터이기 때문에 마지막 평가를 위해 훈련셋을 전처리한다. 

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
count,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000
mean,3.876298,28.619065,5.432607,1.096174,1425.437742,3.030518,35.624783,-119.563008,2.072499
std,1.906783,12.600999,2.523415,0.489529,1143.062756,6.441340,2.136552,2.005251,1.156915
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.750000,32.540000,-124.350000,0.149990
25%,2.566625,18.000000,4.447644,1.005760,785.000000,2.428016,33.930000,-121.800000,1.200000
50%,3.548500,29.000000,5.234243,1.048720,1166.000000,2.817937,34.250000,-118.490000,1.802500
75%,4.747575,37.000000,6.059008,1.100000,1724.000000,3.283243,37.710000,-118.000000,2.647000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,599.714286,41.950000,-114.310000,5.000010


### Q2)  커스텀 변환기  만들자. 

In [6]:
import numpy as np 
#1.지정된 속성의 값을 반올림해서 정수로 바꾸는 함수  
def  round_number(df):
    res =  ['AveRooms','AveBedrms','AveOccup']
    for  c in res:
        df[c] = np.round(df[c])
    return df

#2.표준편차의 2배이상의 값을 제거   : 평균값과 중앙값이 가깝게 나타나기 때문에 도출 값 제거기능 
def  std_exclude(df):
    columns  =  df [['MedInc','AveRooms','Population', 'AveOccup']].columns     
    for col in columns:
        mean = df[col].mean();
        std  = df[col].std(); 
        b = np.abs(df[col]-mean) / std
        df = df[(b < 2 )]     
    return df 


#3. 지역의 인구  600~ 3000명에 대한 평가 및 카테고리 분류 
def category(df):    
    if df < 600:
        return 'few'
    elif df > 3000:
        return  'many'
    else:
        return  'usually'
    
#4. 1~3까지 사용자 함수를 결합한 사용자 정의 함수 설정  
def  custom_conversion(dataframe):    
    df = dataframe.copy()
    df = round_number(df) ##1번 함수를 통해서 정수화  

    ### 조건 데이터 추출 
    df =df[df['HouseAge']< 52 ]
    
    df = df [df['Price'] < 5  ]
    df = std_exclude(df)  ##2번 함수를 통해서 조건에 추출
    
    ### 평균 방 수와 평균 침실 수를 비교   
    df['Bedrms_per_Rooms'] = df['AveBedrms'] / df ['AveRooms']
    df['Population_Feature'] = df['Population'].apply(category)  ## 3번 함수를 적용
    
    ### 카테고리 속성을 더미 변수화 하기  
    feature_dummies  = pd.get_dummies( df['Population_Feature'] , drop_first = True)
    df = pd.concat([df, feature_dummies] , axis =1 )
    
    ####X를 설명변수 , y를 목적변수에 대입해 둔다. 
    X = df.drop(['AveBedrms','Price', 'Population_Feature'  ] , axis =1)
    y = df['Price']
    
    return X,y  
#5. 변환기 실행
X,y   =  custom_conversion(train_set)  

In [7]:
#6.X 통계 확인 
X.describe()

,MedInc,HouseAge,AveRooms,Population,AveOccup,Latitude,Longitude,Bedrms_per_Rooms
count,13490.000000,13490.000000,13490.000000,13490.000000,13490.000000,13490.000000,13490.000000,13490.000000
mean,3.521691,27.732765,5.152780,1323.445219,2.931579,35.603957,-119.501109,0.207191
std,1.336221,11.243943,1.137455,722.577783,0.756020,2.156848,1.981430,0.054826
min,0.643300,1.000000,1.000000,6.000000,1.000000,32.540000,-124.300000,0.000000
25%,2.500000,18.000000,4.000000,805.000000,2.000000,33.930000,-121.550000,0.166667
50%,3.382000,28.000000,5.000000,1183.000000,3.000000,34.240000,-118.450000,0.200000
75%,4.437500,36.000000,6.000000,1695.000000,3.000000,37.680000,-117.990000,0.250000
max,6.831700,51.000000,10.000000,3814.000000,5.000000,41.950000,-114.550000,1.000000


In [8]:
#7. 설명변수 스케일링 작업  
from sklearn.preprocessing import StandardScaler

scaler  = StandardScaler()
X_s = scaler.fit_transform(X)


### Q3.     1.  캐글 데이터를 사용해서 복잡한(비선형) 기계 학습 모델을 사용하여 훈련 할 수 있다. 
      - SVM, RandomForest, GradientBoostingTree ,  MLP  

In [9]:
#1. 모델 적용 하기전 선형회귀 모델에서 예측 오차를 확인 해보자.
from sklearn.linear_model import LinearRegression
from sklearn.metrics  import  mean_squared_error
#선형회귀 모형 생성 
model = LinearRegression().fit(X_s,y)

#모델 예측값  
pred   = model .predict(X_s)

#오차확인  mean_squared_error
mse   =   mean_squared_error(y, pred)

#오차결과를 sqrt로 지정 출력  
rmse  =  np.sqrt(mse)
rmse    #5만 7천달러의 오차를 가진다.  

0.5782170375200296

#### 3-1  SVM 사용
    - 중소규모 데이터 셋 활용할 수 있는 모형
    - 분류, 회귀, 이상치 검출 등에 사용된다. 
    - 하이퍼파라미터 설정  

In [10]:
from sklearn.svm   import SVR

# 모델 생성 및 실행  
svm_model  = SVR()
svm_model.fit(X_s,y)

### 예측값과 정답 라벨의 오차를 구해보자.  

#모델 예측값  
svm_pred   = svm_model .predict(X_s)

#오차확인  mean_squared_error
svm_mse   =   mean_squared_error(y, svm_pred)

#오차결과를 sqrt로 지정 출력  
svm_rmse  =  np.sqrt(svm_mse)
svm_rmse    #4만 9천달러의 오차를 가진다.  

0.49938331668750263

#### 3-2 sklearn.ensemble.RandomForestRegressor
    - 가장 강력한 알고리즘 것 중 하나  
    - 분류, 회귀, 구성요소 - 결정트리 
    - 기계학습 모델을 앙상블 학습에 의해 조합한 모델이다.
    - 앙상블 학습은 여러모델을 훈련하고 회귀일 경우 개별 예측 결과를 평균하여 최종 예측값으로 출력 한다. 

In [11]:
from sklearn.ensemble  import RandomForestRegressor

# 모델 생성 및 실행  
forest_model  = RandomForestRegressor(n_estimators=100, random_state= 12)
forest_model.fit(X_s,y)

### 예측값과 정답 라벨의 오차를 구해보자.  

#모델 예측값  
forest_pred   = forest_model .predict(X_s)

#오차확인  mean_squared_error
forest_mse   =   mean_squared_error(y, forest_pred)

#오차결과를 sqrt로 지정 출력  
forest_rmse  =  np.sqrt(forest_mse)
forest_rmse    #1만 6천달러의 오차를 가진다.  -> 훈련 데이터 셋에 대한 정밀도가 좋다. 

0.16190002162205516

#### 3-3 sklearn.ensemble.GradientBoostingRegressor
    - 경사부스팅  : 결정트리를 순차적으로 훈련하고 모델수정하려고 시도하면서 예측을 한다. 
    - 결정 트리 1의 예측값과 정답 라벨의 오차 
      -> 결정트리 2의 목적 변수로 대입한 후  훈련된 결정트리2의 예측값과 정답 라벨의 오차
      -> 결정트리 3의 목적 변수로 대입한 후  훈련된 결정트리3의 예측값과 정답 라벨의 오차을 그 다음 트리로 전달
      -> 결정트리 n이 끝나면 최종으로  1,2,3,n의 예측의 합을 리턴한다.  

In [12]:
from sklearn.ensemble  import GradientBoostingRegressor

# 모델 생성 및 실행  
gb_model  = GradientBoostingRegressor(random_state= 12)
gb_model.fit(X_s,y)

### 예측값과 정답 라벨의 오차를 구해보자.  

#모델 예측값  
gb_pred   = gb_model .predict(X_s)

#오차확인  mean_squared_error
gb_mse   =   mean_squared_error(y, gb_pred)

#오차결과를 sqrt로 지정 출력  
gb_rmse  =  np.sqrt(gb_mse)
gb_rmse    #4만 5천달러의 오차를 가진다. 

0.45871046190894127

#### 3-4  sklearn.neural_network.MLPRegressor
    - 다층 퍼셉트론  : 여러 인공 신경망(뉴런)의 숨겨진 레이어로 구성된 심층 학습 모델  
    - 뉴런  : 동물의 대뇌 피질에서 발견되는 세포를 말하며  컴퓨터로 표현해서 인공뉴런
    - 인공 뉴럴 네트워크(ANN)  : 인공 뉴런이 네트 워크로 접속된 형태  
    
    - 단층 퍼셉트론 (퍼셉트론)  =  입력뉴런   +  출력 뉴런  
    - 다층 퍼셉트론  = 입력  + 히든  + 출력  
    
    - 신경망에서는 입력값이  0~ 1까지의 범위에 들어가 있는 데이터를 전제로 한다.  -> 최소 ,최대값 스케일링 작업 !!!
    

In [13]:
from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()
X_m = min_max.fit_transform(X)

In [14]:
from sklearn.neural_network import  MLPRegressor
# 모델 생성 및 실행  
mlp_model  = MLPRegressor(max_iter=300, random_state= 12)
mlp_model.fit(X_m,y)

### 예측값과 정답 라벨의 오차를 구해보자.  

#모델 예측값  
mlp_pred   = mlp_model .predict(X_m)

#오차확인  mean_squared_error
mlp_mse   =   mean_squared_error(y, mlp_pred)

#오차결과를 sqrt로 지정 출력  
mlp_rmse  =  np.sqrt(mse)
mlp_rmse    #4만 8천달러의 오차를 가진다. 

0.5782170375200296

In [15]:
#각 모형의 RMSE를 pandas 데이터 프레임으로 저장하고 비교 해보자.  
accuracy = pd.DataFrame( [svm_rmse, forest_rmse, gb_rmse, mlp_rmse]   ,
                         index =['SVM' ,'RandomForest', 'GBoost', 'MLP'], columns =['평가'] )
accuracy

,평가
SVM,0.499383
RandomForest,0.161900
GBoost,0.458710
MLP,0.578217


###  Q4 << 2. 교차 검증을 사용해서 재평가 검증을  할 수 있다.  >>
     - SVM , RandomForest,  GradientBoostingTree ,  MLP   -> 재검증 
     
     - cross_val_score(교차검증) :  훈련데이터를 몇개의 수로 분할 하고, 그 분할된 만큼 훈련, 평가 (평균 제곱오차)를 실시해서
       그 평균치를 모델의 정밀도로 리턴받는다.  
       ex) 5분할 하게 되면 1첫번쨰 서브셋를 평가용으로 제거하고 나머지 4개의 서브셋을 훈련용으로 사용하고 첫번째 학습을 끝낸다
           이작업을 5번- > 5개의 정밀도가 정해진다. -> 정밀도의 평균값 -> 실제의 성능지표 
           https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation

In [16]:
#평가, 평균, 표준편차를 출력하는 사용자 함수를 작성한다. 
def display_scores(scores):
    print('Score :' , scores) 
    print('Mean :' , scores.mean())   
    print('STD : ' , scores.std())    

In [17]:
#1.SVC 재 평가  
from sklearn.model_selection import cross_val_score

svm_scores = cross_val_score(svm_model, X_s, y, scoring='neg_mean_squared_error', cv=5)
svm_rmse_score = np.sqrt(-svm_scores) #-값의 평균 제곱오차가 요구된다. 
display_scores(svm_rmse_score )

Score : [0.51300035 0.50944027 0.52991419 0.51807624 0.51571989]
Mean : 0.5172301870932425
STD :  0.006963547572011202


In [18]:
#2.forest 재 평가  
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_model, X_s, y, scoring='neg_mean_squared_error', cv=5)
forest_rmse_score = np.sqrt(-forest_scores) #-값의 평균 제곱오차가 요구된다. 
display_scores(forest_rmse_score )

Score : [0.44512695 0.43178837 0.45178465 0.42590075 0.44303237]
Mean : 0.43952661760822914
STD :  0.009375054059146277


In [19]:
#3.GradientBoosting  재 평가  
from sklearn.model_selection import cross_val_score

gb_scores = cross_val_score(gb_model, X_s, y, scoring='neg_mean_squared_error', cv=5)
gb_rmse_score = np.sqrt(-gb_scores) #-값의 평균 제곱오차가 요구된다. 
display_scores(gb_rmse_score )

Score : [0.46875213 0.46981314 0.48301373 0.47003838 0.47917313]
Mean : 0.47415810508923395
STD :  0.0058077178420723585


In [20]:
#4.MLP 재 평가  
from sklearn.model_selection import cross_val_score

mlp_scores = cross_val_score(mlp_model, X_m, y, scoring='neg_mean_squared_error', cv=5)
mlp_rmse_score = np.sqrt(-mlp_scores) #-값의 평균 제곱오차가 요구된다. 
display_scores(mlp_rmse_score )

Score : [0.49299609 0.49536545 0.5109027  0.49626535 0.50255921]
Mean : 0.4996177582430844
STD :  0.0064679108092251325


In [21]:
# 각 모델의 검증 결과를 데이터 프레임에 저장한다. mean()
accuracy['CrossValidation'] =  [svm_rmse_score.mean(), \
                                forest_rmse_score.mean(), gb_rmse_score.mean(), mlp_rmse_score.mean() ]
accuracy

,평가,CrossValidation
SVM,0.499383,0.517230
RandomForest,0.161900,0.439527
GBoost,0.458710,0.474158
MLP,0.578217,0.499618


  ###  Q5  << 3.  학습된 모델과 교차검증 후  그리드 검색 및 랜덤 검색을  할 수 있다.   >> 
    - GridSearchCV / RandomizedSearchCV  : 하이퍼파라미터값을  각 모델의 속성에 맞게  즉 생성자 매개인자의 조합을 대입
    
    -  하이퍼파라미터를 이용해서 각각의 조합으로 교차 검증하고 최고의 조합을 리턴해 준다. 
    
    - 두 객체의 차이는 하이퍼 파라미터 값의 검색 영역이 적거나 많거나의 차이이다.  

#### 5-1 SVR의 하이퍼 파라미터   

In [ ]:
from sklearn.model_selection import  GridSearchCV #모델,하이퍼파라미터그룹 , 검증횟수, 평가설정  
#C = 3, gamma= 2, kernel = 2,  cv=5 -> 12번 훈련 * 5 ->  60회 훈련 평가  -> 12가지의 검증 평균값이 출력된다.  
param_grid  =[
              {  'C' : [0.0001, 0.01, 0.1] ,     #1 , 10  
                'gamma' : ['scale', 'auto'],
                'kernel' :['linear', 'rbf' ]}
             ]    
svm_model  = SVR()
svm_grid_search = GridSearchCV(svm_model,param_grid,cv=5,scoring='neg_mean_squared_error' ) 
svm_grid_search.fit(X_s, y)

In [ ]:
#최상의 값 리턴 
svm_grid_search.best_params_  # C:10   -> 0.468 

In [ ]:
# 모형을 통해 오류값 확인  
svm_grid_pred  = svm_grid_search.best_estimator_.predict(X_s)
svm_grid_mse  = mean_squared_error(y,svm_grid_pred )
svm_grid_rmse = np.sqrt(svm_grid_mse)  # RMSE : 평균 제곱의 제곱근 오차 
svm_grid_rmse #0.468  

#### 5-2 랜덤포레스트의 하이퍼 파라미터   

In [ ]:
param_grid =  [
      { 'max_depth' :  [3,5,7,13], 
        'n_estimators' :  [300,400,500] , 
        'random_state'  :[123]}     
]
forest_model = RandomForestRegressor() 
forest_grid_search = GridSearchCV(forest_model,param_grid,cv=5,scoring='neg_mean_squared_error' ) 
forest_grid_search.fit(X_s, y)

In [ ]:
# print(help(RandomForestRegressor ))
forest_grid_search.best_params_

In [ ]:
# 모형을 통해 오류값 확인 RMSE평가하기  
forest_grid_pred  = forest_grid_search.best_estimator_.predict(X_s)
forest_grid_mse  = mean_squared_error(y,forest_grid_pred )
forest_grid_rmse = np.sqrt(forest_mse)  # RMSE : 평균 제곱의 제곱근 오차 
forest_grid_rmse #0.468 

#### 5-3 GradientBoosting의 하이퍼 파라미터   

In [ ]:
param_grid =  [
      { 'max_depth' :  [3,5,7,13], 
        'n_estimators' :  [300,400,500] , 
        'random_state'  :[123]}     
]
gb_model = RandomForestRegressor() 
gb_grid_search = GridSearchCV(gb_model,param_grid,cv=5,scoring='neg_mean_squared_error' ) 
gb_grid_search.fit(X_s, y)

In [ ]:
gb_grid_search.best_params_

In [ ]:
# 모형을 통해 오류값 확인 RMSE평가하기  
gb_grid_pred  = gb_grid_search.best_estimator_.predict(X_s)
gb_grid_mse  = mean_squared_error(y,gb_grid_pred )
gb_grid_rmse = np.sqrt(gb_mse)  # RMSE : 평균 제곱의 제곱근 오차 
gb_grid_rmse #0.468 

#### 5-4 MLP 의 하이퍼 파라미터    _ RandomizedSearchCV 

In [ ]:
from sklearn.model_selection import  RandomizedSearchCV
param_grid =  [
      { 'hidden_layer_sizes' :  [(100),(100,100), (50,50,50),  (20,50,80,20,20)], 
        'max_iter' :  [300,100] , 
        'random_state'  :[123]}     
]
mlp_model = MLPRegressor() 

mlp_random_search = RandomizedSearchCV(mlp_model,
                                       param_grid,
                                       cv=5,
                                       n_jobs=-1,
                                       scoring='neg_mean_squared_error',
                                       random_state=123)
mlp_random_search.fit(X_m, y)

In [ ]:
mlp_random_search.best_params_

In [ ]:
# 모형을 통해 오류값 확인 RMSE평가하기  
mlp_random_pred  = mlp_random_search.best_estimator_.predict(X_m)
mlp_random_mse  = mean_squared_error(y,mlp_random_pred )
mlp_random_rmse = np.sqrt(mlp_random_mse)  # RMSE : 평균 제곱의 제곱근 오차 
mlp_random_rmse #0.468 

In [ ]:
accuracy['Grid_or_Random_Search'] = [svm_grid_rmse,
                                     forest_grid_rmse,
                                     gb_grid_rmse,
                                     mlp_random_rmse]

accuracy

In [ ]:
### Q6)    4. 최고 모델에서 교차 검증 및 최상의 ML 모델로 평가  

### Q6)    4. 최고 모델에서 교차 검증 및 최상의 ML 모델로 평가  

In [ ]:
svm_scores = cross_val_score(svm_grid_search.best_estimator_ , X_s, y,scoring='neg_mean_squared_error',cv=5 )
svm_best_rmse_score = np.sqrt(-svm_scores)
display_scores(svm_best_rmse_score)

In [ ]:
forest_scores = cross_val_score(forest_grid_search.best_estimator_ , X_s, y,scoring='neg_mean_squared_error',cv=5 )
forest_best_rmse_score = np.sqrt(-forest_scores)
display_scores(forest_best_rmse_score)

In [ ]:
gb_scores = cross_val_score(gb_grid_search.best_estimator_ , X_s, y,scoring='neg_mean_squared_error',cv=5 )
gb_best_rmse_score = np.sqrt(-gb_scores)
display_scores(gb_best_rmse_score)

In [ ]:
mlp_scores = cross_val_score(mlp_random_search.best_estimator_ , X_m, y,scoring='neg_mean_squared_error',cv=5 )
mlp_best_rmse_score = np.sqrt(-mlp_scores)
display_scores(mlp_best_rmse_score)

In [ ]:
accuracy['Search_cv'] = [svm_best_rmse_score.mean(),
                                     forest_best_rmse_score.mean(),
                                     gb_best_rmse_score.mean(),
                                     mlp_best_rmse_score.mean()]

accuracy  # 평가전체결과_박민기.png

In [ ]:
## 각 모델의 결과 평균 
accuracy.mean(axis=1).sort_values()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'

In [ ]:
#잔차플롯 확인  -> 상위 2개로 확인  
import seaborn as sns
plt.figure(figsize=(10, 8))
sns.residplot(gb_grid_pred, y - gb_grid_pred)
plt.savefig('gb_train_resid.png');
plt.show()

In [ ]:
X_test, y_test = custom_conversion(test_set)  # 상위 2개 모델

X_test_s = scaler.transform(X_test)

best_pred = gb_grid_search.best_estimator_.predict(X_test_s)
best_mse = mean_squared_error(y_test, best_pred)
best_rmse = np.sqrt(best_mse)
best_rmse

In [ ]:
plt.figure(figsize=(10, 8))
sns.residplot(best_pred, y_test - best_pred)
plt.savefig('gb_test_resid.png')
plt.show()

In [ ]:
import pickle

with open('best_model.pickle', 'wb') as f:
    pickle.dump(gb_grid_search.best_estimator_, f)

In [ ]:
with open('best_model.pickle', 'rb') as f:
    best_model = pickle.load(f)

best_model